# Convoluted Neural Network

In [60]:
''' Needed libraries '''

import numpy as np # For matrix operations and numerical processing
import matplotlib.pyplot as plt # For plotting
import os, sys # For filepaths
import torch # For building the neural network

In [61]:
''' Add the datasets and libraries to the system path '''

# Find the path to our implementations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
home_directory = os.path.dirname(parent_directory)
libraries_path = os.path.join(home_directory, 'Libraries')

# Find the path to the datasets
datasets_path = os.path.join(home_directory, 'Datasets')

# Add them both to the system path
sys.path.append(datasets_path)
sys.path.append(libraries_path)

In [62]:
''' Loading in the training and test sets '''

training_set = np.load(os.path.join(datasets_path, 'fashion_train.npy'))  # Load train set
train_X = training_set[:, :-1] # Define X as all columns except the last one
train_y = training_set[:, -1] # Define y as the last column

test_set = np.load(os.path.join(datasets_path, 'fashion_test.npy'))  # Load test set
test_X = test_set[:, :-1] # Define X as all columns except the last one
test_y = test_set[:, -1] # Define y as the last column


print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(10000, 784)
(10000,)
(5000, 784)
(5000,)


In [63]:
''' Convert the data to tensors '''
tensor_train_X = torch.from_numpy(train_X).float() # Convert to float tensor
tensor_train_y = torch.from_numpy(train_y).long() # Convert to long tensor
tensor_test_X = torch.from_numpy(test_X).float() # Convert to float tensor
tensor_test_y = torch.from_numpy(test_y).long() # Convert to long tensor

In [64]:
''' Create tensors for training and test sets '''
train = torch.utils.data.TensorDataset(tensor_train_X, tensor_train_y) # Create a training set
test = torch.utils.data.TensorDataset(tensor_test_X, tensor_test_y) # Create a test set

In [65]:
''' Define NN architecture '''
class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [66]:
''' Define parameters and initialize the neural network '''
input_size = train_X.shape[1] # Number of features per sample
hidden_size = 1000 # Number of nodes in the hidden layer
number_of_classes = len(np.unique(train_y)) # Number of classes


neural_network = NeuralNetwork(input_size, hidden_size, number_of_classes)

In [89]:
''' Loss function and optimizer '''
loss_function = torch.nn.CrossEntropyLoss() # Cross entropy loss function
# optimizer = torch.optim.Adam(neural_network.parameters(), lr=0.001) # Adam optimizer
# optimizer = torch.optim.SGD(neural_network.parameters(), lr=0.001) # SGD optimizer
# optimizer = torch.optim.SGD(neural_network.parameters(), lr=0.001, momentum=0.9) # SGD optimizer with momentum
# optimizer = torch.optim.SGD(neural_network.parameters(), lr=0.001, momentum=0.9, nesterov=True) # SGD optimizer with momentum and Nesterov momentum
# optimizer = torch.optim.Adagrad(neural_network.parameters(), lr=0.001) # Adagrad optimizer
# optimizer = torch.optim.RMSprop(neural_network.parameters(), lr=0.001) # RMSprop optimizer
optimizer = torch.optim.Adamax(neural_network.parameters(), lr=0.001) # Adamax optimizer

In [90]:
''' Training the neural network '''
batch_size = 100 # Number of samples per batch
num_epochs = 10 # Number of epochs

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True) # Create a train loader

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, input_size) # Reshape the images to vectors
        optimizer.zero_grad() # Zero the gradients
        outputs = neural_network(images) # Forward pass
        loss = loss_function(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass
        optimizer.step() # Update the weights
        if (i+1) % 100 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}')

Epoch: 1/10, Step: 100/100, Loss: 0.2523
Epoch: 2/10, Step: 100/100, Loss: 0.1968
Epoch: 3/10, Step: 100/100, Loss: 0.2645
Epoch: 4/10, Step: 100/100, Loss: 0.1993
Epoch: 5/10, Step: 100/100, Loss: 0.2256
Epoch: 6/10, Step: 100/100, Loss: 0.2551
Epoch: 7/10, Step: 100/100, Loss: 0.1614
Epoch: 8/10, Step: 100/100, Loss: 0.3035
Epoch: 9/10, Step: 100/100, Loss: 0.1769
Epoch: 10/10, Step: 100/100, Loss: 0.1782


In [91]:
''' Testing the neural network '''
neural_network.eval() # Set the model to evaluation mode
correct = 0 # Initialize the number of correct predictions
total = 0 # Initialize the total number of predictions

test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False) # Create a test loader

with torch.no_grad(): # Disable gradient calculation
    for images, labels in test_loader:
        images = images.view(-1, input_size) # Reshape the images to vectors
        outputs = neural_network(images) # Forward pass
        _, predicted = torch.max(outputs.data, 1) # Get the predicted class
        total += labels.size(0) # Add the number of labels
        correct += (predicted == labels).sum().item() # Add the number of correct predictions

print(f'Accuracy of the network on the {len(test_loader)*batch_size} test images: {100*correct/total}%')

Accuracy of the network on the 5000 test images: 84.36%
